In [ ]:
import os
import csv
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

# Set up the environment variable for the API key
os.environ["MISTRAL_API_KEY"] = "FRUPgGYYJtJxjzdhvsl6vhxi2g8ValAj"

# Initialize the Mistral AI client
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"
client = MistralClient(api_key=api_key)

# Define base templates
base_templates = [
    {
        "initial_query": "who is the prime minister of {country}? Answer in 2 lines.",
        "initial_topic": "Politics - {country}",
        "follow_up": [
            {"query": "what are the duties of the prime minister of {country}? Answer in 2 lines.", "topic": "Politics - {country}"},
            {"query": "how long has the current prime minister of {country} been in office? Answer in 2 lines.", "topic": "Politics - {country}"}
        ]
    },
    {
        "initial_query": "who won the {year} world cup in {sport}? Answer in 2 lines.",
        "initial_topic": "Sports - {sport}",
        "follow_up": [
            {"query": "who was the top scorer in the {year} {sport} world cup? Answer in 2 lines.", "topic": "Sports - {sport}"},
            {"query": "when and where is the next {sport} world cup? Answer in 2 lines.", "topic": "Sports - {sport}"}
        ]
    },
    {
        "initial_query": "what is the latest discovery in {field}? Answer in 2 lines.",
        "initial_topic": "Science - {field}",
        "follow_up": [
            {"query": "tell me more about the latest discovery in {field}. Answer in 2 lines.", "topic": "Science - {field}"},
            {"query": "how does this discovery in {field} impact us? Answer in 2 lines.", "topic": "Science - {field}"}
        ]
    },
    {
        "initial_query": "who is the president of {country}? Answer in 2 lines.",
        "initial_topic": "Politics - {country}",
        "follow_up": [
            {"query": "what are the responsibilities of the president of {country}? Answer in 2 lines.", "topic": "Politics - {country}"},
            {"query": "how long has the current president of {country} been in office? Answer in 2 lines.", "topic": "Politics - {country}"}
        ]
    },
    {
        "initial_query": "who won the {year} Nobel Prize in {field}? Answer in 2 lines.",
        "initial_topic": "Awards - {field}",
        "follow_up": [
            {"query": "what was the contribution of the {year} Nobel Prize winner in {field}? Answer in 2 lines.", "topic": "Awards - {field}"},
            {"query": "who were the other notable nominees for the {year} Nobel Prize in {field}? Answer in 2 lines.", "topic": "Awards - {field}"}
        ]
    },
    {
        "initial_query": "what is the capital of {country}? Answer in 2 lines.",
        "initial_topic": "Geography - {country}",
        "follow_up": [
            {"query": "what is the population of {country}'s capital? Answer in 2 lines.", "topic": "Geography - {country}"},
            {"query": "what are some famous landmarks in {country}'s capital? Answer in 2 lines.", "topic": "Geography - {country}"}
        ]
    },
    {
        "initial_query": "what is the most popular food in {country}? Answer in 2 lines.",
        "initial_topic": "Culture - {country}",
        "follow_up": [
            {"query": "what is the origin of the most popular food in {country}? Answer in 2 lines.", "topic": "Culture - {country}"},
            {"query": "how is the most popular food in {country} prepared? Answer in 2 lines.", "topic": "Culture - {country}"}
        ]
    },
    {
        "initial_query": "what are the main tourist attractions in {country}? Answer in 2 lines.",
        "initial_topic": "Tourism - {country}",
        "follow_up": [
            {"query": "what is the best time to visit {country}? Answer in 2 lines.", "topic": "Tourism - {country}"},
            {"query": "what are the travel requirements for visiting {country}? Answer in 2 lines.", "topic": "Tourism - {country}"}
        ]
    },
    {
        "initial_query": "what are the main industries in {country}? Answer in 2 lines.",
        "initial_topic": "Economy - {country}",
        "follow_up": [
            {"query": "what is the GDP of {country}? Answer in 2 lines.", "topic": "Economy - {country}"},
            {"query": "who are the main trading partners of {country}? Answer in 2 lines.", "topic": "Economy - {country}"}
        ]
    },
    {
        "initial_query": "what is the most spoken language in {country}? Answer in 2 lines.",
        "initial_topic": "Language - {country}",
        "follow_up": [
            {"query": "how many people speak the most spoken language in {country}? Answer in 2 lines.", "topic": "Language - {country}"},
            {"query": "what are some common phrases in the most spoken language in {country}? Answer in 2 lines.", "topic": "Language - {country}"}
        ]
    },
    # New Templates
    {
        "initial_query": "what is the latest trend in {field} technology? Answer in 2 lines.",
        "initial_topic": "Technology - {field}",
        "follow_up": [
            {"query": "who are the leading companies in {field} technology? Answer in 2 lines.", "topic": "Technology - {field}"},
            {"query": "what are the potential impacts of the latest trend in {field} technology? Answer in 2 lines.", "topic": "Technology - {field}"}
        ]
    },
    {
        "initial_query": "what was a significant event in the history of {country}? Answer in 2 lines.",
        "initial_topic": "History - {country}",
        "follow_up": [
            {"query": "how did the significant event shape the current status of {country}? Answer in 2 lines.", "topic": "History - {country}"},
            {"query": "who were the key figures in this significant event in {country}? Answer in 2 lines.", "topic": "History - {country}"}
        ]
    },
    {
        "initial_query": "who is the most famous author from {country}? Answer in 2 lines.",
        "initial_topic": "Literature - {country}",
        "follow_up": [
            {"query": "what is the most famous work of the author from {country}? Answer in 2 lines.", "topic": "Literature - {country}"},
            {"query": "what themes are prevalent in the works of this author from {country}? Answer in 2 lines.", "topic": "Literature - {country}"}
        ]
    },
    {
        "initial_query": "what is the most effective treatment for {condition}? Answer in 2 lines.",
        "initial_topic": "Health - {condition}",
        "follow_up": [
            {"query": "what are the symptoms of {condition}? Answer in 2 lines.", "topic": "Health - {condition}"},
            {"query": "how can {condition} be prevented? Answer in 2 lines.", "topic": "Health - {condition}"}
        ]
    },
    {
        "initial_query": "what is the biggest environmental issue facing {country}? Answer in 2 lines.",
        "initial_topic": "Environment - {issue}",
        "follow_up": [
            {"query": "what efforts are being made to address this environmental issue in {country}? Answer in 2 lines.", "topic": "Environment - {issue}"},
            {"query": "what can individuals do to help mitigate this environmental issue in {country}? Answer in 2 lines.", "topic": "Environment - {issue}"}
        ]
    },
    {
        "initial_query": "what is the highest-grossing film in {genre}? Answer in 2 lines.",
        "initial_topic": "Entertainment - {genre}",
        "follow_up": [
            {"query": "who are the main actors in the highest-grossing {genre} film? Answer in 2 lines.", "topic": "Entertainment - {genre}"},
            {"query": "what makes this {genre} film so popular? Answer in 2 lines.", "topic": "Entertainment - {genre}"}
        ]
    },
    {
        "initial_query": "what is the top university in {country}? Answer in 2 lines.",
        "initial_topic": "Education - {country}",
        "follow_up": [
            {"query": "what are the admission requirements for the top university in {country}? Answer in 2 lines.", "topic": "Education - {country}"},
            {"query": "what are the most popular courses at the top university in {country}? Answer in 2 lines.", "topic": "Education - {country}"}
        ]
    },
    {
        "initial_query": "what is the fastest-growing sector in {country}'s economy? Answer in 2 lines.",
        "initial_topic": "Economy - {sector}",
        "follow_up": [
            {"query": "what are the main factors contributing to the growth of this sector in {country}? Answer in 2 lines.", "topic": "Economy - {sector}"},
            {"query": "what challenges does this sector face in {country}? Answer in 2 lines.", "topic": "Economy - {sector}"}
        ]
    },
    {
        "initial_query": "what is the most popular destination in {country}? Answer in 2 lines.",
        "initial_topic": "Travel - {country}",
        "follow_up": [
            {"query": "what activities can tourists enjoy at the most popular destination in {country}? Answer in 2 lines.", "topic": "Travel - {country}"},
            {"query": "what is the best way to reach the most popular destination in {country}? Answer in 2 lines.", "topic": "Travel - {country}"}
        ]
    },
    {
        "initial_query": "what is the significance of {festival} in {country}? Answer in 2 lines.",
        "initial_topic": "Culture - {festival}",
        "follow_up": [
            {"query": "how is {festival} celebrated in {country}? Answer in 2 lines.", "topic": "Culture - {festival}"},
            {"query": "what are the traditional foods associated with {festival} in {country}? Answer in 2 lines.", "topic": "Culture - {festival}"}
        ]
    }
]

variations = {
    "country": ["India", "UK", "USA", "Canada", "Australia", "Germany", "France", "Japan","srilanka"],
    "year": ["2018", "2022", "2014", "2010"],
    "sport": ["football", "cricket", "basketball", "rugby","Formula1"],
    "field": ["space", "medicine", "technology", "environment"],
    "condition": ["diabetes", "hypertension", "asthma", "cancer"],
    "issue": ["climate change", "deforestation", "air pollution", "water scarcity"],
    "genre": ["action", "drama", "comedy", "horror"],
    "festival": ["Diwali", "Christmas", "Eid", "Hanukkah"],
    "sector": ["technology", "healthcare", "finance", "agriculture"]
}


def generate_variations(template, variations):
    from itertools import product
    keys, values = zip(*variations.items())
    combinations = [dict(zip(keys, v)) for v in product(*values)]
    
    templates = []
    for combo in combinations:
        initial_query = template["initial_query"].format(**combo)
        initial_topic = template["initial_topic"].format(**combo)
        
        follow_ups = []
        for follow_up in template["follow_up"]:
            follow_ups.append({
                "query": follow_up["query"].format(**combo),
                "topic": follow_up["topic"].format(**combo)
            })
        
        templates.append({
            "initial_query": initial_query,
            "initial_topic": initial_topic,
            "follow_up": follow_ups
        })
    
    return templates

# Generate all template variations
all_templates = []
for base_template in base_templates:
    all_templates.extend(generate_variations(base_template, variations))

# Ensure each prompt is used only once
used_prompts = set()

def generate_unique_response(prompt):
    if prompt in used_prompts:
        return None
    used_prompts.add(prompt)
    return generate_response(prompt)

def generate_response(prompt):
    messages = [ChatMessage(role="user", content=prompt)]
    chat_response = client.chat(model=model, messages=messages)
    full_response = chat_response.choices[0].message.content
    return full_response

# Generate dataset
dataset = []

for template in all_templates:
    initial_query = template["initial_query"]
    initial_topic = template["initial_topic"]

    # Generate initial query and response
    initial_response = generate_unique_response(initial_query)
    if initial_response:
        print(f"Prompt: {initial_query}\nResponse: {initial_response}\n")
        dataset.append({"user_query": initial_query, "bot_response": initial_response, "expanded_query": "", "topic": initial_topic})

    # Generate follow-up queries and responses
    for follow_up in template["follow_up"]:
        follow_up_query = follow_up["query"]
        follow_up_topic = follow_up["topic"]

        follow_up_response = generate_unique_response(follow_up_query)
        if follow_up_response:
            print(f"Prompt: {follow_up_query}\nResponse: {follow_up_response}\n")
            dataset.append({"user_query": follow_up_query, "bot_response": follow_up_response, "expanded_query": follow_up_query, "topic": follow_up_topic})

# Save dataset to CSV file
csv_file = 'expanded_conversations_dataset.csv'

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["user_query", "bot_response", "expanded_query", "topic"])
    writer.writeheader()
    for data in dataset:
        writer.writerow(data)

print(f"Dataset generation complete. Saved to '{csv_file}'.")
